In [1]:
# imports

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns

from plotly.offline import init_notebook_mode,download_plotlyjs,iplot

import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing dataset

df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')

In [3]:
#Basic analysis and features engineering

#1. Removing Unwanted cloumns and features
try:
    df_train.drop(labels=['Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
    df_test.drop(labels=['Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
except Exception as e:
    pass

In [4]:
#2. Getting information about null values

temp_df = df_train.isnull().sum().reset_index()
temp_df['Percentage'] = (temp_df[0]/len(df_train))*100

temp_df.columns = ['Column Name','Number of null values', 'Null values in percentage']

print(f'The length of dataset is \t {len(df_train)}')

The length of dataset is 	 8523


In [5]:
#3. Making correction in 'Item_Fat_content' column

def convert(x):
    if x in ['low fat','LF']:
        return 'Low Fat'
    elif x=='reg':
        return 'Regular'
    else:
        return x
    
df_train['Item_Fat_Content'] = df_train['Item_Fat_Content'].apply(convert)

df_test['Item_Fat_Content'] = df_test['Item_Fat_Content'].apply(convert)

print(f'Now Unique values in this column in Train set are \t {df_train["Item_Fat_Content"].unique()}')
print(f'Now unique values in this cloumn in test set are \t {df_test["Item_Fat_Content"].unique()}')

Now Unique values in this column in Train set are 	 ['Low Fat' 'Regular']
Now unique values in this cloumn in test set are 	 ['Low Fat' 'Regular']


In [6]:
#4 Dealing with missing values in categorical type column ie 'Outlet_size'

count = df_train['Outlet_Size'].value_counts().reset_index()
count.iplot(kind='bar',color='deepskyblue',x='index',y='Outlet_Size',
            title='High VS Medium VS Small',xTitle='Size',yTitle='Frequency')

In [7]:
# Removing missing values from medium in both training and test

df_train['Outlet_Size'].fillna(value='Medium',inplace=True)
df_test['Outlet_Size'].fillna(value='Medium',inplace=True)

In [8]:
#Creating feature matrix and target vector

x_train = df_train.iloc[:,:-1].values
y_train = df_train.iloc[:,-1].values

x_test = df_test.values

In [9]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

In [10]:
imputer = Imputer()
x_train[:,[0]] = imputer.fit_transform(x_train[:,[0]])
x_test[:,[0]] = imputer.fit_transform(x_test[:,[0]])

c:\users\micha\appdata\local\conda\conda\envs\machinelearning\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning:

Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.



In [11]:
#2 Dealing with categorical values in features /cloumns 

labelencoder_x = LabelEncoder()
x_train[:,1] = labelencoder_x.fit_transform(x_train[:,1])
x_train[:,3] = labelencoder_x.fit_transform(x_train[:,3])
x_train[:,5] = labelencoder_x.fit_transform(x_train[:,5])
x_train[:,6] = labelencoder_x.fit_transform(x_train[:,6])
x_train[:,7] = labelencoder_x.fit_transform(x_train[:,7])

onehotencoder_x = OneHotEncoder(categorical_features=[3,5,6,7])
x_train = onehotencoder_x.fit_transform(x_train).toarray()

x_test[:,1] = labelencoder_x.fit_transform(x_test[:,1])
x_test[:,3] = labelencoder_x.fit_transform(x_test[:,3])
x_test[:,5] = labelencoder_x.fit_transform(x_test[:,5])
x_test[:,6] = labelencoder_x.fit_transform(x_test[:,6])
x_test[:,7] = labelencoder_x.fit_transform(x_test[:,7])


#need to be done when we have more than twocategorical values

onehotencoder_x = OneHotEncoder(categorical_features=[3,5,6,7])
x_test = onehotencoder_x.fit_transform(x_test).toarray()

c:\users\micha\appdata\local\conda\conda\envs\machinelearning\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning:

The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.

c:\users\micha\appdata\local\conda\conda\envs\machinelearning\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning:

The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.



In [12]:
#apply Feature scaling on feature matrix

sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.fit_transform(x_test)

In [14]:
x_train

array([[-0.28685487, -0.17419336, -0.11434598, ..., -0.73814723,
        -0.97073217,  1.74745381],
       [-0.28685487, -0.17419336, -0.11434598, ...,  1.35474328,
        -0.90811123, -1.48902325],
       [-0.28685487, -0.17419336, -0.11434598, ..., -0.73814723,
        -0.95691733,  0.01004021],
       ...,
       [-0.28685487, -0.17419336, -0.11434598, ..., -0.73814723,
        -0.59978449, -0.89720755],
       [-0.28685487, -0.17419336, -0.11434598, ...,  1.35474328,
         1.53287976, -0.60797692],
       [-0.28685487, -0.17419336, -0.11434598, ..., -0.73814723,
        -0.41193591, -1.05226104]])

In [15]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [22]:
model = Sequential()
model.add(Dense(20,input_dim=30,kernel_initializer='normal',activation='relu'))
model.add(Dense(1,kernel_initializer='normal'))
model.compile(loss='mean_absolute_percentage_error',optimizer='adam')

In [24]:
model.fit(x_train,y_train,validation_split=0.2,epochs=200)

Train on 6818 samples, validate on 1705 samples
Epoch 1/200
6818/6818 [==============================] - 1s 178us/step - loss: 54.9901 - val_loss: 53.8421
Epoch 2/200
6818/6818 [==============================] - 1s 121us/step - loss: 54.6859 - val_loss: 53.5704
Epoch 3/200
6818/6818 [==============================] - 1s 152us/step - loss: 54.3979 - val_loss: 53.3452
Epoch 4/200
6818/6818 [==============================] - 1s 149us/step - loss: 54.1215 - val_loss: 53.1059
Epoch 5/200
6818/6818 [==============================] - 1s 138us/step - loss: 53.8550 - val_loss: 52.8983
Epoch 6/200
6818/6818 [==============================] - 1s 119us/step - loss: 53.5947 - val_loss: 52.6710
Epoch 7/200
6818/6818 [==============================] - 1s 123us/step - loss: 53.3473 - val_loss: 52.4318
Epoch 8/200
6818/6818 [==============================] - 1s 117us/step - loss: 53.1153 - val_loss: 52.2020
Epoch 9/200
6818/6818 [==============================] - 1s 120us/step - loss: 52.8744 - val_los

6818/6818 [==============================] - 1s 117us/step - loss: 45.7592 - val_loss: 45.3510
Epoch 77/200
6818/6818 [==============================] - 1s 117us/step - loss: 45.7356 - val_loss: 45.3360
Epoch 78/200
6818/6818 [==============================] - 1s 118us/step - loss: 45.7121 - val_loss: 45.3083
Epoch 79/200
6818/6818 [==============================] - 1s 116us/step - loss: 45.6954 - val_loss: 45.3140
Epoch 80/200
6818/6818 [==============================] - 1s 120us/step - loss: 45.6727 - val_loss: 45.2674
Epoch 81/200
6818/6818 [==============================] - 1s 125us/step - loss: 45.6561 - val_loss: 45.2684
Epoch 82/200
6818/6818 [==============================] - 1s 115us/step - loss: 45.6307 - val_loss: 45.2487
Epoch 83/200
6818/6818 [==============================] - 1s 115us/step - loss: 45.6148 - val_loss: 45.2541
Epoch 84/200
6818/6818 [==============================] - 1s 124us/step - loss: 45.5937 - val_loss: 45.2207
Epoch 85/200
6818/6818 [=================

6818/6818 [==============================] - 1s 116us/step - loss: 44.7929 - val_loss: 44.8619
Epoch 152/200
6818/6818 [==============================] - 1s 120us/step - loss: 44.7852 - val_loss: 44.8684
Epoch 153/200
6818/6818 [==============================] - 1s 117us/step - loss: 44.7832 - val_loss: 44.8583
Epoch 154/200
6818/6818 [==============================] - 1s 118us/step - loss: 44.7654 - val_loss: 44.8545
Epoch 155/200
6818/6818 [==============================] - 1s 118us/step - loss: 44.7623 - val_loss: 44.8513
Epoch 156/200
6818/6818 [==============================] - 1s 123us/step - loss: 44.7535 - val_loss: 44.8598
Epoch 157/200
6818/6818 [==============================] - 1s 118us/step - loss: 44.7425 - val_loss: 44.8361
Epoch 158/200
6818/6818 [==============================] - 1s 116us/step - loss: 44.7376 - val_loss: 44.8383
Epoch 159/200
6818/6818 [==============================] - 1s 118us/step - loss: 44.7314 - val_loss: 44.8310
Epoch 160/200
6818/6818 [========

In [25]:
y_pred = model.predict(x_test)

y_pred[:5]

array([[1173.8251],
       [1012.3114],
       [ 433.6255],
       [1784.8308],
       [3496.5862]], dtype=float32)